# Import des librairies

In [1]:
#librairie pour lire les données
import pandas as pd

#librairie pour machine learning

#permet de faire le tfidf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
#permet de separer les données en données d'entrainement et de test
from sklearn.model_selection import train_test_split
#algos
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
#permet d'encapsuler les étapes d'entrainement de l'algo
from sklearn.pipeline import Pipeline
#les mesures des performances de l'algo
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score

#librairie for nlp
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import FrenchStemmer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gbjs0748\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gbjs0748\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\gbjs0748\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Travail sur les données 

In [2]:
#lire le dataset
df = pd.read_csv('intent.csv',encoding= 'latin1',sep=';')
df.head()

,phrases,intention,class
0,activer un renvoi d'appel,renvoi d'appel,1
1,demande à djingo pro de renvoyer les appels,renvoi d'appel,1
2,de renvoyer les appels,renvoi d'appel,1
3,aide moi à renvoyer mes appels,renvoi d'appel,1
4,je veux renvoyer mes appels,renvoi d'appel,1


In [3]:
#je fais un nouveau tableau avec lequel je vais travailler
df_intents = df[['phrases','class']]

In [4]:
#enlève les accents et les apstrophes

phrases = df_intents.phrases
phrases = phrases.str.replace('[^A-z]',' ').str.replace(' +', ' ').str.strip()
#print(phrases)

In [5]:
#j'enlève les stop words
stopW = stopwords.words('french')
#print(stopW)
#exclude stopswords
df_intents['phrases_sans_stop_words'] = df_intents['phrases'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopW)]))

In [6]:
df_intents.head()

,phrases,class,phrases_sans_stop_words
0,activer un renvoi d'appel,1,activer renvoi d'appel
1,demande à djingo pro de renvoyer les appels,1,demande djingo pro renvoyer appels
2,de renvoyer les appels,1,renvoyer appels
3,aide moi à renvoyer mes appels,1,aide renvoyer appels
4,je veux renvoyer mes appels,1,veux renvoyer appels


In [7]:
# on fait un stemmer : stemmer permet de ne garder que la racine d'un mot
def steammer_phrase(phrases):
    
    stemmer = FrenchStemmer()
    tokens = word_tokenize(phrases)
    tokens = [stemmer.stem(word) for word in tokens]
    return ' '.join(tokens)

In [8]:
#on ajoute une colonne de phrases que l'on a stemmé
df_intents['stemmed_phrases'] = df_intents['phrases_sans_stop_words'].apply(lambda phrases: steammer_phrase(phrases))

In [9]:
df_intents.head()

,phrases,class,phrases_sans_stop_words,stemmed_phrases
0,activer un renvoi d'appel,1,activer renvoi d'appel,activ renvoi d'appel
1,demande à djingo pro de renvoyer les appels,1,demande djingo pro renvoyer appels,demand djingo pro renvoi appel
2,de renvoyer les appels,1,renvoyer appels,renvoi appel
3,aide moi à renvoyer mes appels,1,aide renvoyer appels,aid renvoi appel
4,je veux renvoyer mes appels,1,veux renvoyer appels,veux renvoi appel


## Transformation des données en vecteur 

## TFIDF

le TFIDF est une approche bag-of-words (bow) permettant de représenter les mots d’un document à l’aide d’une matrice de nombres. Le terme bow signifie que l’ordre des mots dans la phrase n’est pas pris en compte, contrairement à des approches plus poussées de Deep Learning (word embeddings : word2vec, GLoVE). Néanmoins, TFIDF est une méthode efficace utilisée dans de nombreux outils de querying puisqu’elle donne de l’importance aux mots qui apparaissent de temps en temps mais pas trop, tout en en limitant l’importance des mots qui apparaissent souvent.

TFIDF: Cette mesure statistique permet d'évaluer l'importance d'un terme contenu dans un document, relativement à une collection ou un corpus.Le poids augmente proportionnellement au nombre d'occurrences du mot dans le document. 
Term Frequency : Frequence du terme dans le document mot/tous les mots
Inverse document Frequency: TFIDF = Term Frequency * Inverse document frequency : log(doc/nbre de doc dans lequel apparait le terme)

## On utilise un pipeline de l'algo logistic regression

Le pipeline va transformer le texte en vecteur et appliquer une regression logistique aux données

In [23]:
#un pipeline avec une logistic regression
tfidf_vectorizer = TfidfVectorizer()
lr = LogisticRegression()
tfidf_lr_pipe = Pipeline([('tfidf', tfidf_vectorizer), ('lr',lr)])



In [24]:
# definir le Xtrain et le y_train 
X_train, X_test, y_train, y_test = train_test_split(df_intents['stemmed_phrases'], df_intents['class'], test_size = 0.3)

In [12]:
#print(X_train)
#print(y_train)
#print(X_test)

In [25]:
tfidf_lr_pipe.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('lr',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=

In [26]:
y_pred = tfidf_lr_pipe.predict(X_test)

In [27]:
#l'accuracy du model
acc = accuracy_score(y_test,y_pred)

In [28]:
print(acc)

0.8888888888888888


In [29]:
#la matrice de confusion
cm = confusion_matrix(y_test,y_pred)
print(cm)

[[2 0 0]
 [0 4 0]
 [1 0 2]]


In [30]:
print(y_pred)

[3 2 3 1 1 1 2 2 2]


In [31]:
#print(y_test)

In [32]:
#new data  
new_X = ['renvoi les appels']
result= tfidf_lr_pipe.predict(new_X)
print(result)

[1]
